Hey, I'm Jobert Gutierrez and hereafter you'll find the logic and code used to answer the fourth assignment in the program Data Engineering Zoomcamp offered by Data Talks Club.

# __Module 4 Homework: Analytics Engineering__

For this homework, you will need the following datasets:

- [Green Taxi dataset (2019 and 2020)](https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/green)
- [Yellow Taxi dataset (2019 and 2020)](https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/yellow)
- [For Hire Vehicle dataset (2019)](https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/fhv)

Before you start,

1. Make sure you, at least, have them in GCS with a External Table OR a Native Table - use whichever method you prefer to accomplish that (Workflow Orchestration with pandas-gbq, dlt for gcs, dlt for BigQuery, gsutil, etc)
2. You should have exactly `7,778,101` records in your Green Taxi table
3. You should have exactly `109,047,518` records in your Yellow Taxi table
4. You should have exactly `43,244,696` records in your FHV table
5. Build the staging models for green/yellow as shown in [here](https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/04-analytics-engineering/taxi_rides_ny/models/staging)

Note: If you don't have access to GCP, you can spin up a local Postgres instance and ingest the datasets above

### __Question 1.Understanding dbt model resolution__
Provided you've got the following sources.yaml?

In [ ]:
version: 2

sources:
  - name: raw_nyc_tripdata
    database: "{{ env_var('DBT_BIGQUERY_PROJECT', 'dtc_zoomcamp_2025') }}"
    schema:   "{{ env_var('DBT_BIGQUERY_SOURCE_DATASET', 'raw_nyc_tripdata') }}"
    tables:
      - name: ext_green_taxi
      - name: ext_yellow_taxi

with the following env variables setup where dbt runs:

In [ ]:
export DBT_BIGQUERY_PROJECT=myproject
export DBT_BIGQUERY_DATASET=my_nyc_tripdata

What does this .sql model compile to?

In [ ]:
select * 
from {{ source('raw_nyc_tripdata', 'ext_green_taxi' ) }}

--select * from dtc_zoomcamp_2025.raw_nyc_tripdata.ext_green_taxi <br>
--select * from dtc_zoomcamp_2025.my_nyc_tripdata.ext_green_taxi <br>
--select * from myproject.raw_nyc_tripdata.ext_green_taxi <br>
--select * from myproject.my_nyc_tripdata.ext_green_taxi <br>
-- select * from dtc_zoomcamp_2025.raw_nyc_tripdata.green_taxi

### Answer: 
The indicated sql model compiles to `select * from myproject.raw_nyc_tripdata.ext_green_taxi`.

### __Question 2.dbt Variables & Dynamic Models__

Say you have to modify the following dbt_model (`fct_recent_taxi_trips.sql`) to enable Analytics Engineers to dynamically control the date range.

- In development, you want to process only the last 7 days of trips
- In production, you need to process the last 30 days for analytics


In [ ]:
select *
from {{ ref('fact_taxi_trips') }}
where pickup_datetime >= CURRENT_DATE - INTERVAL '30 days'

What would you change to accomplish that in a such way that command line arguments takes precedence over ENV_VARs, which takes precedence over DEFAULT value?

- Add `ORDER BY pickup_datetime DESC` and `LIMIT {{ var("days_back", 30) }}`
- Update the WHERE clause to `pickup_datetime >= CURRENT_DATE - INTERVAL '{{ var("days_back", 30) }}' DAY`
- Update the WHERE clause to `pickup_datetime >= CURRENT_DATE - INTERVAL '{{ env_var("DAYS_BACK", "30") }}' DAY`
- Update the WHERE clause to `pickup_datetime >= CURRENT_DATE - INTERVAL '{{ var("days_back", env_var("DAYS_BACK", "30")) }}' DAY`
- Update the WHERE clause to `pickup_datetime >= CURRENT_DATE - INTERVAL '{{ env_var("DAYS_BACK", var("days_back", "30")) }}' DAY`

### Answer: 
The right answer is __updating the WHERE clause to `pickup_datetime >= CURRENT_DATE - INTERVAL '{{ var("days_back", env_var("DAYS_BACK", "30")) }}' DAY`__ because this option:
- ensures prioritization If days_back is passed via --vars, 
- it falls back to env_var("DAYS_BACK", "30") if `days_back` isn't set, and 
- set the dafault value of 30 if neither is set.

### __Question 3.dbt Data Lineage and Execution__

Considering the data lineage below and that taxi_zone_lookup is the only materialization build (from a .csv seed file):

![homework_q2](homework_q2.png "Diagram")

Select the option that does NOT apply for materializing fct_taxi_monthly_zone_revenue:

- dbt run
- dbt run --select +models/core/dim_taxi_trips.sql+ --target prod
- dbt run --select +models/core/fct_taxi_monthly_zone_revenue.sql
- dbt run --select +models/core/
- dbt run --select models/staging/+

### Answer:
The right answer is `dbt run --select +models/core/dim_taxi_trips.sql+ --target prod` because this will not materialize the table in dev. It will create the table in production only.

### __Question 4.__
dbt Macros and Jinja
Consider you're dealing with sensitive data (e.g.: [PII](https://en.wikipedia.org/wiki/Personal_data)), that is only available to your team and very selected few individuals, in the `raw layer` of your DWH (e.g: a specific BigQuery dataset or PostgreSQL schema),

- Among other things, you decide to obfuscate/masquerade that data through your staging models, and make it available in a different schema (a `staging layer`) for other Data/Analytics Engineers to explore

- And optionally, yet another layer (`service layer`), where you'll build your dimension (dim_) and fact (fct_) tables (assuming the [Star Schema dimensional modeling](https://www.databricks.com/glossary/star-schema)) for Dashboarding and for Tech Product Owners/Managers

You decide to make a macro to wrap a logic around it:

In [ ]:
{% macro resolve_schema_for(model_type) -%}

    {%- set target_env_var = 'DBT_BIGQUERY_TARGET_DATASET'  -%}
    {%- set stging_env_var = 'DBT_BIGQUERY_STAGING_DATASET' -%}

    {%- if model_type == 'core' -%} {{- env_var(target_env_var) -}}
    {%- else -%}                    {{- env_var(stging_env_var, env_var(target_env_var)) -}}
    {%- endif -%}

{%- endmacro %}

And use on your staging, dim_ and fact_ models as:

In [ ]:
{{ config(
    schema=resolve_schema_for('core'), 
) }}

That all being said, regarding macro above, select all statements that are true to the models using it:

- Setting a value for DBT_BIGQUERY_TARGET_DATASET env var is mandatory, or it'll fail to compile
- Setting a value for DBT_BIGQUERY_STAGING_DATASET env var is mandatory, or it'll fail to compile
- When using core, it materializes in the dataset defined in DBT_BIGQUERY_TARGET_DATASET
- When using stg, it materializes in the dataset defined in DBT_BIGQUERY_STAGING_DATASET, or defaults to DBT_BIGQUERY_TARGET_DATASET
- When using staging, it materializes in the dataset defined in DBT_BIGQUERY_STAGING_DATASET, or defaults to DBT_BIGQUERY_TARGET_DATASET

### Answer:
The right options are: 
- `Setting a value for DBT_BIGQUERY_TARGET_DATASET env var is mandatory, or it'll fail to compile` because the macro always references env_var(target_env_var), which is DBT_BIGQUERY_TARGET_DATASET.

- `When using core, it materializes in the dataset defined in DBT_BIGQUERY_TARGET_DATASET` because if the sets `model_type == 'core'`, the macro resolves to `{{- env_var(target_env_var) -}}`.

- `When using staging, it materializes in the dataset defined in DBT_BIGQUERY_STAGING_DATASET, or defaults to DBT_BIGQUERY_TARGET_DATASET` because if model_type is not 'core', the macro executes the else clause meaning that if DBT_BIGQUERY_STAGING_DATASET exists, it uses it or it falls back to DBT_BIGQUERY_TARGET_DATASET otherwise.

## __Serious SQL__
Alright, in module 1, you had a SQL refresher, so now let's build on top of that with some serious SQL.

These are not meant to be easy - but they'll boost your SQL and Analytics skills to the next level.
So, without any further do, let's get started...

You might want to add some new dimensions year (e.g.: 2019, 2020), quarter (1, 2, 3, 4), year_quarter (e.g.: 2019/Q1, 2019-Q2), and month (e.g.: 1, 2, ..., 12), extracted from pickup_datetime, to your fct_taxi_trips OR dim_taxi_trips.sql models to facilitate filtering your queries

### __Question 5.Taxi Quarterly Revenue Growth__
1. Create a new model `fct_taxi_trips_quarterly_revenue.sql`
2. Compute the Quarterly Revenues for each year for based on total_amount
3. Compute the Quarterly YoY (Year-over-Year) revenue growth
- e.g.: In 2020/Q1, Green Taxi had -12.34% revenue growth compared to 2019/Q1
- e.g.: In 2020/Q4, Yellow Taxi had +34.56% revenue growth compared to 2019/Q4

Considering the YoY Growth in 2020, which were the yearly quarters with the best (or less worse) and worst results for green, and yellow

- green: {best: 2020/Q2, worst: 2020/Q1}, yellow: {best: 2020/Q2, worst: 2020/Q1}
- green: {best: 2020/Q2, worst: 2020/Q1}, yellow: {best: 2020/Q3, worst: 2020/Q4}
- green: {best: 2020/Q1, worst: 2020/Q2}, yellow: {best: 2020/Q2, worst: 2020/Q1}
- green: {best: 2020/Q1, worst: 2020/Q2}, yellow: {best: 2020/Q1, worst: 2020/Q2}
- green: {best: 2020/Q1, worst: 2020/Q2}, yellow: {best: 2020/Q3, worst: 2020/Q4}

### Answer:
I created the `fct_taxi_trips_quarterly_revenue.sql` model as indicated bellow:

In [ ]:
{{
    config(
        materialized='table'
    )
}}
with trips_data as (
    select * from {{ ref("fact_trips")}}
)
select
-- revenue grouping
pickup_zone as revenue_zone,
concat('Q', extract(quarter from pickup_datetime), '_', extract(year from pickup_datetime)) as quarter_label,
{{ dbt.date_trunc("quarter","pickup_datetime")}} as revenue_quarter,

service_type, 

-- Revenue calculation 
sum(total_amount) as total_revenue_quarterly

from trips_data
group by 1,2,3,4

Then computed the quarterly YoY revenue in Big Query as:

In [ ]:
WITH revenue_data AS (
    SELECT 
        service_type,
        quarter_label,
        revenue_quarter,
        SUM(total_revenue_quarterly) AS total_revenue_quarterly,
        -- Get revenue from the same quarter last year
        LAG(SUM(total_revenue_quarterly)) OVER (
            PARTITION BY service_type 
            ORDER BY revenue_quarter
        ) AS previous_year_revenue
    FROM `taxi-data-24-dtc.dbt_jgutierrez.fct_taxi_trips_quarterly_revenue`
    GROUP BY quarter_label, service_type, revenue_quarter
)
SELECT 
    service_type,
    quarter_label,
    revenue_quarter,
    total_revenue_quarterly,
    previous_year_revenue,
    -- Avoid division by zero
    ROUND(
        (total_revenue_quarterly - previous_year_revenue) / NULLIF(previous_year_revenue, 0) * 100, 2
    ) AS yoy_growth_percentage
FROM revenue_data
WHERE date(revenue_quarter) between '2019-01-01' and '2021-01-01'
ORDER BY revenue_quarter, service_type;


Then, the right answer is __green: {best: 2020/Q1, worst: 2020/Q2}, yellow: {best: 2020/Q1, worst: 2020/Q2}__.

### __Question 6. P97/P95/P90 Taxi Monthly Fare__
1. Create a new model `fct_taxi_trips_monthly_fare_p95.sql`
2. Filter out invalid entries (fare_amount > 0, trip_distance > 0, and payment_type_description in ('Cash', 'Credit Card'))
3. Compute the continous percentile of fare_amount partitioning by service_type, year and and month

Now, what are the values of p97, p95, p90 for Green Taxi and Yellow Taxi, in April 2020?

- green: {p97: 55.0, p95: 45.0, p90: 26.5}, yellow: {p97: 52.0, p95: 37.0, p90: 25.5}
- green: {p97: 55.0, p95: 45.0, p90: 26.5}, yellow: {p97: 31.5, p95: 25.5, p90: 19.0}
- green: {p97: 40.0, p95: 33.0, p90: 24.5}, yellow: {p97: 52.0, p95: 37.0, p90: 25.5}
- green: {p97: 40.0, p95: 33.0, p90: 24.5}, yellow: {p97: 31.5, p95: 25.5, p90: 19.0}
- green: {p97: 55.0, p95: 45.0, p90: 26.5}, yellow: {p97: 52.0, p95: 25.5, p90: 19.0}


### Answer:
I created the model `fct_taxi_trips_monthly_fare_p95.sql` using the following code:

In [ ]:
{{
    config(
        materialized='table'
    )
}}

WITH trips_data AS (
    SELECT 
        service_type,
        EXTRACT(YEAR FROM pickup_datetime) AS trip_year,
        EXTRACT(MONTH FROM pickup_datetime) AS trip_month,
        fare_amount,
        trip_distance,
        payment_type_description,

        -- Compute the continuous percentile of fare_amount
        PERCENT_RANK() OVER (
            PARTITION BY service_type, EXTRACT(YEAR FROM pickup_datetime), EXTRACT(MONTH FROM pickup_datetime)
            ORDER BY fare_amount
        ) AS fare_percentile

    FROM {{ ref('fact_trips') }}

    -- Filter out invalid data
    WHERE fare_amount > 0
      AND trip_distance > 0
      AND payment_type_description IN ('Cash', 'Credit Card')
)

SELECT *
FROM trips_data
ORDER BY service_type, trip_year, trip_month, fare_percentile

Then, I calculated the requested information using big query as: 

In [ ]:
SELECT
    service_type,
    -- trip_year,
    -- trip_month,
    PERCENTILE_CONT(fare_amount, 0.97) OVER (PARTITION BY service_type, trip_year, trip_month) AS fare_p97,
    PERCENTILE_CONT(fare_amount, 0.95) OVER (PARTITION BY service_type, trip_year, trip_month) AS fare_p95,
    PERCENTILE_CONT(fare_amount, 0.90) OVER (PARTITION BY service_type, trip_year, trip_month) AS fare_p90
FROM `taxi-data-24-dtc.dbt_jgutierrez.fct_taxi_trips_monthly_fare_p95`
WHERE trip_year= 2020 AND trip_month =4

The option that properly answer the question is __green: {p97: 55.0, p95: 45.0, p90: 26.5}, yellow: {p97: 31.5, p95: 25.5, p90: 19.0}__.

### __Question 7.Top #Nth longest P90 travel time Location for FHV__

Prerequisites:

- Create a staging model for FHV Data (2019), and DO NOT add a deduplication step, just filter out the entries where `where dispatching_base_num is not null`
- Create a core model for FHV Data (`dim_fhv_trips.sql`) joining with `dim_zones`. Similar to what has been done [here](https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/04-analytics-engineering/taxi_rides_ny/models/staging/stg_green_tripdata.sql)
- Add some new dimensions `year` (e.g.: 2019) and `month` (e.g.: 1, 2, ..., 12), based on `pickup_datetime`, to the core model to facilitate filtering for your queries

Now: 

1. Create a new model `fct_fhv_monthly_zone_traveltime_p90.sql`
2. For each record in `dim_fhv_trips.sql`, compute the [timediff](https://cloud.google.com/bigquery/docs/reference/standard-sql/time_functions#time_diff), in seconds between dropoff_datetime and pickup_datetime - we'll call it `trip_duration` for this exercise
Compute the continous `p90` of `trip_duration` partitioning by year, month, pickup_location_id, and dropoff_location_id

For the Trips that respectively started from `Newark Airport`, `SoHo`, and `Yorkville East`, in November 2019, what are dropoff_zones with the 2nd longest p90 trip_duration ?

- East Village, Chinatown, Garment District
- East Village, Park Slope, Clinton East
- East Village, Saint Albans, Howard Beach
- East Village, Rosedale, Bath Beach
- East Village, Yorkville East, Greenpoint


### Answer:
The data is stored in a __GCP Bucket__ for the external table.